In [1]:
pip install dash plotly


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.comNote: you may need to restart the kernel to use updated packages.



In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# Load the datasets
df = pd.read_csv("tsdataset.csv")  # Telangana dataset
df2 = pd.read_csv("apdatase.csv")  # Andhra Pradesh dataset
df3 = pd.read_csv("tndataset.csv")  # Tamil Nadu dataset
df4 = pd.read_csv("kldataset.csv")  # Kerala dataset
df5 = pd.read_csv("goadataset.csv")  # Goa dataset
df6 = pd.read_csv("deldataset.csv")  # Delhi dataset

# Normalize column names to remove any extra spaces
for dataset in [df, df2, df3, df4, df5, df6]:
    dataset.columns = dataset.columns.str.strip()

# Print column names to debug
datasets_info = {
    "Telangana": df,
    "Andhra Pradesh": df2,
    "Tamil Nadu": df3,
    "Kerala": df4,
    "Goa": df5,
    "Delhi": df6
}
for name, dataset in datasets_info.items():
    print(f"Columns in {name} Dataset: {dataset.columns.tolist()}")

# Initialize the Dash app
app = dash.Dash(__name__)

# Layout of the dashboard
app.layout = html.Div([
    html.H1("Educational Analytics Dashboard", style={"textAlign": "center"}),

    # Dropdown for selecting a dataset (TS, AP, TN, KL, Goa, Delhi datasets)
    html.Div([
        html.Label("Select Dataset:"),
        dcc.Dropdown(
            id="dataset-dropdown",
            options=[
                {"label": "Telangana", "value": "tsdataset"},
                {"label": "Andhra Pradesh", "value": "apdatase"},
                {"label": "Tamil Nadu", "value": "tndataset"},
                {"label": "Kerala", "value": "kldataset"},
                {"label": "Goa", "value": "goadataset"},
                {"label": "Delhi", "value": "deldataset"}  # Added Delhi dataset
            ],
            value="tsdataset",  # Default value
            clearable=False
        )
    ], style={"width": "50%", "margin": "auto"}),

    # Dropdown for selecting a location
    html.Div([
        html.Label("Select Location:"),
        dcc.Dropdown(
            id="location-dropdown",
            options=[],  # Will be populated dynamically based on the dataset selection
            value=None,
            clearable=False
        )
    ], style={"width": "50%", "margin": "auto"}),

    # Graph components
    html.Div([
        dcc.Graph(id="total-enrollment-bar"),
        dcc.Graph(id="urban-rural-pie"),
        dcc.Graph(id="school-category-bar"),
        dcc.Graph(id="education-level-line"),
        dcc.Graph(id="school-management-bar")
    ])
])

# Callbacks to update graphs and location options dynamically
@app.callback(
    [
        Output("location-dropdown", "options"),
        Output("total-enrollment-bar", "figure"),
        Output("urban-rural-pie", "figure"),
        Output("school-category-bar", "figure"),
        Output("education-level-line", "figure"),
        Output("school-management-bar", "figure"),
    ],
    [Input("dataset-dropdown", "value"), Input("location-dropdown", "value")]
)
def update_graphs_and_locations(selected_dataset, selected_location):
    try:
        # Load the selected dataset
        if selected_dataset == "tsdataset":
            df_combined = df
        elif selected_dataset == "apdatase":
            df_combined = df2
        elif selected_dataset == "tndataset":
            df_combined = df3
        elif selected_dataset == "kldataset":
            df_combined = df4
        elif selected_dataset == "goadataset":
            df_combined = df5
        else:  # Delhi dataset
            df_combined = df6

        # Debugging: Print the shape and columns to verify the dataset structure
        print(f"Selected Dataset: {selected_dataset}")
        print(f"Dataset Shape: {df_combined.shape}")
        print(f"Dataset Columns: {df_combined.columns.tolist()}")

        # Ensure the columns are created properly before accessing
        df_combined["Primary_Total"] = df_combined["Primary (I-V)"]
        df_combined["Secondary_Total"] = df_combined["Secondary (IX-X)"]
        df_combined["HigherSecondary_Total"] = df_combined["Higher Secondary (XI-XII)"]

        # Debugging: Ensure that the new columns are added
        print(f"Columns after adding totals: {df_combined.columns.tolist()}")

        # Generate location options based on the selected dataset
        location_options = [{"label": loc, "value": loc} for loc in df_combined["Location"].unique()]

        # If no location is selected, default to the first one
        if not selected_location:
            selected_location = df_combined["Location"].unique()[0]

        # Filter data for the selected location
        filtered_df = df_combined[df_combined["Location"] == selected_location]

        # Debugging: Check if filtered data is empty
        if filtered_df.empty:
            print(f"No data found for the selected location: {selected_location}")
            return location_options, {}, {}, {}, {}, {}

        # 1. Total Enrollment Bar Chart
        total_enrollment_fig = px.bar(
            x=["Primary", "Secondary", "Higher Secondary"],
            y=[
                filtered_df["Primary_Total"].sum(),
                filtered_df["Secondary_Total"].sum(),
                filtered_df["HigherSecondary_Total"].sum(),
            ],
            labels={"x": "Education Level", "y": "Total Enrollment"},
            title="Total Enrollment by Education Level"
        )

        # 2. Urban vs Rural Pie Chart
        urban_rural_fig = px.pie(
            filtered_df,
            names="Rural/Urban",
            values="Primary_Total",
            title="Urban vs Rural Enrollment Distribution"
        )

        # 3. School Category Bar Chart
        category_fig = px.bar(
            filtered_df,
            x="School Category",
            y="Primary_Total",
            title="Enrollment by School Category"
        )

        # 4. Education Level Line Chart
        level_trends = filtered_df.melt(
            id_vars=["Location", "School Category"],
            value_vars=["Primary_Total", "Secondary_Total", "HigherSecondary_Total"],
            var_name="Education Level",
            value_name="Total Students"
        )
        line_fig = px.line(
            level_trends,
            x="Education Level",
            y="Total Students",
            color="School Category",
            title="Education Level Trends by School Category"
        )

        # 5. School Management Bar Chart
        management_fig = px.bar(
            filtered_df,
            x="School Management",
            y="Primary_Total",
            title="Enrollment by School Management"
        )

        return location_options, total_enrollment_fig, urban_rural_fig, category_fig, line_fig, management_fig

    except Exception as e:
        print(f"Error in callback: {e}")
        return [], {}, {}, {}, {}, {}

# Run the Dash app
if __name__ == "__main__":
    app.run_server(debug=True)


Columns in Telangana Dataset: ['Location', 'Rural/Urban', 'School Category', 'School Management', 'Primary (I-V)', 'Secondary (IX-X)', 'Higher Secondary (XI-XII)']
Columns in Andhra Pradesh Dataset: ['Location', 'Rural/Urban', 'School Category', 'School Management', 'Primary (I-V)', 'Secondary (IX-X)', 'Higher Secondary (XI-XII)']
Columns in Tamil Nadu Dataset: ['Location', 'Rural/Urban', 'School Category', 'School Management', 'Primary (I-V)', 'Secondary (IX-X)', 'Higher Secondary (XI-XII)']
Columns in Kerala Dataset: ['Location', 'Rural/Urban', 'School Category', 'School Management', 'Primary (I-V)', 'Secondary (IX-X)', 'Higher Secondary (XI-XII)']
Columns in Goa Dataset: ['Location', 'Rural/Urban', 'School Category', 'School Management', 'Primary (I-V)', 'Secondary (IX-X)', 'Higher Secondary (XI-XII)']
Columns in Delhi Dataset: ['Location', 'Rural/Urban', 'School Category', 'School Management', 'Primary (I-V)', 'Secondary (IX-X)', 'Higher Secondary (XI-XII)']
